In [482]:
# For the purpose of this project, I used a dataset I found in Kaggle, called "Fashion-MNIST" which is a slight
# variation to the commonly known MNIST dataset. I decided to use the "Fashion-MNIST" dataset just to differ from 
# the others hehe.

In [503]:
# Here we import all the need libraries

import pandas as pd
import numpy as np
import heapq
import timeit

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import classification_report

In [484]:
# Load the datasets from the FashionMNIST folder.

data_train = pd.read_csv("./FashionMNIST/fashion-mnist_train.csv")
data_test = pd.read_csv("./FashionMNIST/fashion-mnist_test.csv")

In [485]:
# Representation of how the dataset looks like.

data_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [488]:
# Split the train and test dataframes into features(X) and labels(Y)

X_train = data_train.drop(["label"],axis=1)
Y_train = data_train[["label"]].values.ravel()
X_test = data_test.drop(["label"],axis=1)
Y_test = data_test[["label"]].values.ravel()

# I used .ravel() due to some inconsistences with the dataset (..Thanks StackOverflow :D)

# Also, since all features range from 0 to 255, we don't need to normalize them.

In [489]:
# We can look at how many rows and columns each dataset has

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [490]:
# Function to create different kind of sizes for training datasets (For testing purposes)

def create_dataset(size):
    """makes a dataset of size "size", and returns that datasets images and targets
    This is used to make the dataset that will be stored by a model and used in 
    experimenting with different stored dataset sizes
    """
    small_x_train = X_train[:size]
    small_y_train = Y_train[:size]
    
    return small_x_train, small_y_train


In [497]:
# Smaller train set of 1000 rows

small_x_train, small_y_train = create_dataset(5000)

small_x_train.shape, small_y_train.shape

((5000, 784), (5000,))

# K-Nearest Neighbors Method

In [504]:
# Function to find the most frequent element in a list

def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

# Function to calculate the K-NN method accuracy

def k_nn(k, x_test, y_test, x_train, y_train, distance_metric):
    
    start = timeit.default_timer()
    
    # Calculate the distance between each image in train set and test set
    # I have used cosine similarity, manhattan or euclidean distance
    
    if(distance_metric == "cosine"):
        cosine = cosine_similarity(x_test, x_train)
        dist = cosine
    elif(distance_metric == "manh"):
        manh = manhattan_distances(x_test, x_train)
        dist = manh
    elif(distance_metric == "eucl"):
        eucl = euclidean_distances(x_test, x_train)
        dist = eucl
    
    # Here is use a heap/priority queue to store in an array the indices of the nearest neighbors
    # For cosine similarity, I return the indices of the k neighbors with the LARGEST similarity
    # For euclidean or manhattan distance, I return the indices of the k neighbors with the SMALLEST distance
    
    if(distance_metric == "cosine"):
        knn = [(heapq.nlargest((k), range(len(i)), i.take)) for i in dist]
    elif(distance_metric == "eucl" or distance_metric == "manh"):
        knn = [(heapq.nsmallest((k), range(len(i)), i.take)) for i in dist]
    
    # Here i convert indices to labels
    
    knn = [[y_train[j] for j in i[:k]] for i in knn]
    
    # Here we choose the most frequent label as the predicted label for every image of the test set
    
    pred = [(most_frequent(i)) for i in knn]
    pred = np.array(pred)
    
    # We report the accuracy, f1 score, precision and recall for each of the label classes (0-9)
    
    print(classification_report(y_test, pred))
    
    stop = timeit.default_timer()
    
    print('Time: ', stop - start)

### RESULTS AND COMPARISONS:


---
- **3 KNN**
- **5.000 Rows**
- **Euclidean Distance**

In [510]:
k_nn(3,X_test,Y_test,small_x_train,small_y_train,"eucl")

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      1000
           1       0.98      0.95      0.96      1000
           2       0.71      0.72      0.72      1000
           3       0.88      0.83      0.85      1000
           4       0.72      0.74      0.73      1000
           5       0.99      0.71      0.83      1000
           6       0.56      0.56      0.56      1000
           7       0.80      0.92      0.86      1000
           8       0.97      0.91      0.94      1000
           9       0.84      0.95      0.89      1000

    accuracy                           0.81     10000
   macro avg       0.82      0.81      0.81     10000
weighted avg       0.82      0.81      0.81     10000

Time:  44.390025000000605


---
- **3 KNN**
- **60.000 Rows**
- **Euclidean Distance**

In [524]:
k_nn(3,X_test,Y_test,X_train,Y_train,"eucl")

              precision    recall  f1-score   support

           0       0.78      0.85      0.81      1000
           1       0.98      0.97      0.98      1000
           2       0.76      0.79      0.78      1000
           3       0.91      0.89      0.90      1000
           4       0.80      0.79      0.79      1000
           5       1.00      0.82      0.90      1000
           6       0.66      0.61      0.64      1000
           7       0.88      0.94      0.91      1000
           8       0.98      0.96      0.97      1000
           9       0.88      0.97      0.92      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Time:  563.1084115999984


---
- **3 KNN**
- **60.000 Rows**
- **Cosine Similarity**

In [525]:
k_nn(3,X_test,Y_test,X_train,Y_train,"cosine")

              precision    recall  f1-score   support

           0       0.77      0.85      0.81      1000
           1       0.99      0.99      0.99      1000
           2       0.79      0.79      0.79      1000
           3       0.92      0.89      0.91      1000
           4       0.77      0.85      0.81      1000
           5       1.00      0.78      0.87      1000
           6       0.72      0.61      0.66      1000
           7       0.88      0.94      0.91      1000
           8       0.98      0.97      0.98      1000
           9       0.85      0.97      0.91      1000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000

Time:  895.7321326999954


---
- **1 KNN**
- **5.000 Rows**
- **Euclidean Distance**

In [511]:
k_nn(1,X_test,Y_test,small_x_train,small_y_train,"eucl")

              precision    recall  f1-score   support

           0       0.74      0.78      0.75      1000
           1       0.97      0.95      0.96      1000
           2       0.70      0.71      0.70      1000
           3       0.86      0.80      0.83      1000
           4       0.70      0.71      0.71      1000
           5       0.98      0.73      0.84      1000
           6       0.53      0.55      0.54      1000
           7       0.82      0.91      0.86      1000
           8       0.97      0.92      0.95      1000
           9       0.84      0.95      0.89      1000

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.80     10000
weighted avg       0.81      0.80      0.80     10000

Time:  39.031778899996425


---
- **1 KNN**
- **60.000 Rows**
- **Euclidean Distance**

In [515]:
k_nn(1,X_test,Y_test,X_train,Y_train,"eucl")

              precision    recall  f1-score   support

           0       0.77      0.83      0.80      1000
           1       0.98      0.98      0.98      1000
           2       0.75      0.77      0.76      1000
           3       0.89      0.88      0.88      1000
           4       0.78      0.76      0.77      1000
           5       0.99      0.86      0.92      1000
           6       0.64      0.62      0.63      1000
           7       0.90      0.95      0.93      1000
           8       0.98      0.95      0.97      1000
           9       0.90      0.97      0.94      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Time:  525.5209583999967


---
- **1 KNN**
- **60.000 Rows**
- **Cosine Similarity**

In [516]:
k_nn(1,X_test,Y_test,X_train,Y_train,"cosine")

              precision    recall  f1-score   support

           0       0.77      0.82      0.80      1000
           1       0.98      0.99      0.98      1000
           2       0.77      0.79      0.78      1000
           3       0.92      0.89      0.91      1000
           4       0.77      0.81      0.79      1000
           5       0.99      0.82      0.90      1000
           6       0.70      0.62      0.65      1000
           7       0.90      0.94      0.92      1000
           8       0.98      0.97      0.97      1000
           9       0.86      0.97      0.91      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Time:  828.915626099988


# Nearest Class Centroid Method

In [508]:
# Function to calculate the NCC method accuracy

def NCC(classes, x_test, y_test, x_train, y_train, distance_metric):
    
    start = timeit.default_timer()
    
    # Array of 9 classes x 784 feature vectors 
    centers= np.zeros((classes,x_train.shape[1]))
    
    # Array of how many images in each class
    centersNo= [0 for i in range(10)]

    # Calculate the class centroids 
    
    for i in range(x_train.shape[0]):
        centers[y_train[i]]+=x_train.iloc[i]
        centersNo[y_train[i]]+=1
        
    for i in range(classes):
        centers[i]/=centersNo[i]
        
     
    # Down below, the code is almost identical to the knn above, but instead of the training set we have the class centroids
    # and we only need the 1st "nearest neighbor", thus the 1st closest class centroid
    # Calculate the distance between each image in train set and test set
    # I have used cosine similarity, manhattan or euclidean distance
    
    if(distance_metric == "cosine"):
        cosine = cosine_similarity(x_test, centers)
        dist = cosine
    elif(distance_metric == "manh"):
        manh = manhattan_distances(x_test, centers)
        dist = manh
    elif(distance_metric == "eucl"):
        eucl = euclidean_distances(x_test, centers)
        dist = eucl
        
    
    # Here is use a heap/priority queue to store in an array the index of the nearest centroid
    # For cosine similarity, I return the indx of the centroid with the LARGEST similarity
    # For euclidean or manhattan distance, I return the indices of the k neighbors with the SMALLEST distance
    
    if(distance_metric == "cosine"):
        knn = [(heapq.nlargest((1), range(len(i)), i.take)) for i in dist]
    elif(distance_metric == "eucl" or distance_metric == "manh"):
        knn = [(heapq.nsmallest((1), range(len(i)), i.take)) for i in dist]
    
    
    # Here we choose the most frequent label as the predicted label for every image of the test set
    
    pred = [i for i in knn]
    pred = np.array(pred)
    
    # We report the accuracy, f1 score, precision and recall for each of the label classes (0-9)
    
    print(classification_report(y_test, pred))
    
    stop = timeit.default_timer()
    
    print('Time: ', stop - start)
        

### Results and Comparisons

---
- **5.000 Rows**
- **Cosine Similarity**

In [518]:
NCC(10,X_test,Y_test,small_x_train,small_y_train,"cosine")

              precision    recall  f1-score   support

           0       0.71      0.77      0.74      1000
           1       0.97      0.91      0.94      1000
           2       0.57      0.64      0.60      1000
           3       0.69      0.88      0.77      1000
           4       0.52      0.51      0.51      1000
           5       0.54      0.29      0.38      1000
           6       0.39      0.26      0.31      1000
           7       0.67      0.81      0.73      1000
           8       0.92      0.83      0.87      1000
           9       0.67      0.84      0.75      1000

    accuracy                           0.68     10000
   macro avg       0.66      0.68      0.66     10000
weighted avg       0.66      0.68      0.66     10000

Time:  8.229619499994442


---
- **5.000 Rows**
- **Euclidean Distance**

In [519]:
NCC(10,X_test,Y_test,small_x_train,small_y_train,"eucl")

              precision    recall  f1-score   support

           0       0.71      0.70      0.71      1000
           1       0.97      0.90      0.93      1000
           2       0.60      0.50      0.55      1000
           3       0.71      0.81      0.76      1000
           4       0.54      0.61      0.57      1000
           5       0.49      0.76      0.60      1000
           6       0.37      0.20      0.26      1000
           7       0.74      0.78      0.76      1000
           8       0.94      0.76      0.84      1000
           9       0.84      0.86      0.85      1000

    accuracy                           0.69     10000
   macro avg       0.69      0.69      0.68     10000
weighted avg       0.69      0.69      0.68     10000

Time:  1.4389641000016127


---
- **5.000 Rows**
- **Manhattan Distance**

In [520]:
NCC(10,X_test,Y_test,small_x_train,small_y_train,"manh")

              precision    recall  f1-score   support

           0       0.77      0.62      0.69      1000
           1       0.76      0.95      0.84      1000
           2       0.64      0.26      0.37      1000
           3       0.56      0.68      0.61      1000
           4       0.43      0.66      0.52      1000
           5       0.43      0.55      0.48      1000
           6       0.38      0.17      0.23      1000
           7       0.58      0.88      0.70      1000
           8       0.96      0.55      0.70      1000
           9       0.84      0.79      0.82      1000

    accuracy                           0.61     10000
   macro avg       0.63      0.61      0.60     10000
weighted avg       0.63      0.61      0.60     10000

Time:  1.4846222000051057


---
- **60.000 Rows**
- **Cosine Similarity**

In [521]:
NCC(10,X_test,Y_test,X_train,Y_train,"cosine")

              precision    recall  f1-score   support

           0       0.70      0.77      0.73      1000
           1       0.97      0.91      0.94      1000
           2       0.57      0.66      0.61      1000
           3       0.69      0.88      0.77      1000
           4       0.53      0.52      0.52      1000
           5       0.56      0.28      0.37      1000
           6       0.40      0.25      0.31      1000
           7       0.65      0.83      0.73      1000
           8       0.92      0.85      0.88      1000
           9       0.68      0.84      0.75      1000

    accuracy                           0.68     10000
   macro avg       0.67      0.68      0.66     10000
weighted avg       0.67      0.68      0.66     10000

Time:  310.6555013999896


---
- **60.000 Rows**
- **Euclidean Distance**

In [522]:
NCC(10,X_test,Y_test,X_train,Y_train,"eucl")

              precision    recall  f1-score   support

           0       0.71      0.70      0.70      1000
           1       0.96      0.90      0.92      1000
           2       0.59      0.50      0.54      1000
           3       0.71      0.80      0.75      1000
           4       0.55      0.60      0.57      1000
           5       0.48      0.76      0.59      1000
           6       0.37      0.20      0.26      1000
           7       0.74      0.79      0.76      1000
           8       0.93      0.76      0.84      1000
           9       0.85      0.85      0.85      1000

    accuracy                           0.69     10000
   macro avg       0.69      0.69      0.68     10000
weighted avg       0.69      0.69      0.68     10000

Time:  14.73205340000277


---
- **60.000 Rows**
- **Manhattan Distance**

In [523]:
NCC(10,X_test,Y_test,X_train,Y_train,"manh")

              precision    recall  f1-score   support

           0       0.77      0.62      0.69      1000
           1       0.76      0.94      0.84      1000
           2       0.64      0.25      0.36      1000
           3       0.56      0.69      0.61      1000
           4       0.43      0.66      0.52      1000
           5       0.42      0.53      0.47      1000
           6       0.37      0.17      0.23      1000
           7       0.57      0.89      0.69      1000
           8       0.96      0.55      0.70      1000
           9       0.85      0.79      0.82      1000

    accuracy                           0.61     10000
   macro avg       0.63      0.61      0.59     10000
weighted avg       0.63      0.61      0.59     10000

Time:  15.737960799990105


# CONCLUSION
---
Based on the experiments above, for a 10.000 rows test set:

***The most accurate kNN algorithm (86% accuracy)*** uses:

- **3 KNN**
- **The whole Training Set**
- **Euclidean Distance**

With almost ***9 minutes*** of training and predicting time

***The most accurate NCC algorithm (69% accuracy)*** uses:

- **The whole Training Set OR 5000 rows on Training Set**
- **Euclidean Distance**

With almost ***15 seconds*** for the whole Training Set and ***1.5 second*** for only 5000 rows on Training Set.

## So, between kNN and NCC, we have:
---
***Most Accurate:*** ***kNN (86%)*** vs NCC (69%)

***Fastest (60.000 rows):*** kNN (9 minutes) vs ***NCC (15 seconds)***